In [1]:
import pandas as pd
# from dotenv import load_dotenv, get_key
import os

In [2]:
tables_path = '../Other_Files/Таблицы'
tables = [filename for filename in os.listdir(tables_path) if os.path.isfile(f'{tables_path}/{filename}')]

In [3]:
tables

['Cemetery.xlsx',
 'Education.xlsx',
 'Health.xlsx',
 'Heritage.xlsx',
 'HeritageArea.xlsx',
 'Social.xlsx',
 'Sport.xlsx']

In [ ]:
# df = pd.read_excel(f'{tables_path}/{tables[0]}')[['CLASSID', 'NAME', 'ADDRESS', 'STATUS', 'REG_STATUS']]

In [ ]:
# df

,CLASSID,NAME,ADDRESS,STATUS,REG_STATUS
0,602050301,Кладбище №2,Вурнарское шоссе и подъездные пути к кладбищу,1,4
1,602050301,Кладбище №3,Карачуры-1,1,4
2,602050301,Клабище №4,Карачуры-2,1,4
3,602050301,Кладбище №5,Карачуры-3,1,4
4,602050301,Кладбище №6,"г. Чебоксары, ул. Энтузиастов",1,4
5,602050301,Кладбище №7,"г. Чебоксары, ул. Мичммана Павлова",1,4
6,602050301,Кладбище №8,"г. Чебоксары, ул. Калинина",1,4
7,602050301,Кладбище №9,"ГО Чебоксары, пос. Восточный",1,4
8,602050301,Кладбище №10,"г. Чебоксары, пр-т Тракторостроителей",1,4
9,602050301,Альгешевское кладбище,"ГО Чебоксары, пос. Альгешево (Лапсарская промз...",1,4


In [4]:
tables_dfs = {}

for table in tables:
    df = pd.read_excel(f'{tables_path}/{table}')
    df = df[['CLASSID', 'NAME', 'ADDRESS', 'STATUS', 'REG_STATUS']] if 'REG_STATUS' in df.columns else df[['CLASSID', 'NAME', 'ADDRESS', 'STATUS']]

    tables_dfs[table.split('.xlsx')[0]] = df

In [5]:
list(tables_dfs.keys())

['Cemetery',
 'Education',
 'Health',
 'Heritage',
 'HeritageArea',
 'Social',
 'Sport']

---

In [11]:
def join_dfs(df_name: str) -> pd.DataFrame:
    statuses = {
        1: 'существующий',
        2: 'планируемый',
        3: 'планируемый к реконструкции',
        4: 'планируемый к ликвидации'
    }
    
    try:
        df = tables_dfs[df_name]

        values = pd.read_excel(f'{tables_path}/Values/{df_name}.xlsx')[1:].rename(columns={'Код объекта' : 'CLASSID'})

        joined = df.set_index('CLASSID').join(values[values.columns[:2]].set_index('CLASSID'))
        
        joined = joined.reset_index()
        joined['STATUS'] = joined['STATUS'].replace(statuses)


        # print(df)
        return(joined)


    except:     # Ловим ошибку, когда нет файлов
        return False


In [ ]:
dfs_joined = {}

for key, df in tables_dfs.items():
    res = join_dfs(key)
    print(key, True if not isinstance(res, bool) else res)

    dfs_joined[key] = res


Cemetery True
Education True
Health True
Heritage True
HeritageArea True
Social True
Sport True


In [49]:
for key, df in dfs_joined.items():
    print(f"|{key}|         {'Все ок' if None not in df['Значение'].values else 'ЕСТЬ ПУСТЫЕ ЗНАЧЕНИЯ'}")

|Cemetery|         Все ок
|Education|         Все ок
|Health|         Все ок
|Heritage|         Все ок
|HeritageArea|         Все ок
|Social|         Все ок
|Sport|         Все ок


In [44]:
None in dfs_joined[list(dfs_joined.keys())[0]]['Значение'].copy().replace({'Кладбище' : None}).values

True

In [45]:
None in dfs_joined[list(dfs_joined.keys())[0]]['Значение'].copy().values

False